# Merge EV Sales and CO2 tables

In [1]:
import pandas as pd

In [3]:
co = pd.read_csv("clean_co2.csv")
ev = pd.read_csv("clean_ev_sales.csv")

In [7]:
co.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
ev.drop("Unnamed: 0", axis=1, inplace=True)

In [10]:
co

,country,year,co2
0,Afghanistan,1950,0.084272
1,Afghanistan,1951,0.091600
2,Afghanistan,1952,0.091600
3,Afghanistan,1953,0.106256
4,Afghanistan,1954,0.106256
...,...,...,...
16699,Global,2017,36096.739276
16700,Global,2018,36826.506600
16701,Global,2019,37082.558969
16702,Global,2020,35264.085734


In [11]:
ev

,country,year,value
0,Australia,2011,4.900000e+01
1,Australia,2011,3.900000e-04
2,Australia,2011,6.500000e-03
3,Australia,2011,4.900000e+01
4,Australia,2012,2.200000e+02
...,...,...,...
3639,USA,2023,1.700000e-01
3640,USA,2023,9.700000e+03
3641,USA,2023,2.900000e+05
3642,USA,2023,1.300000e+06


In [13]:
merged = pd.merge(ev, co, on=["country", "year"], how="left")
merged

,country,year,value,co2
0,Australia,2011,4.900000e+01,403.828773
1,Australia,2011,3.900000e-04,403.828773
2,Australia,2011,6.500000e-03,403.828773
3,Australia,2011,4.900000e+01,403.828773
4,Australia,2012,2.200000e+02,406.150910
...,...,...,...,...
3639,USA,2023,1.700000e-01,NaN
3640,USA,2023,9.700000e+03,NaN
3641,USA,2023,2.900000e+05,NaN
3642,USA,2023,1.300000e+06,NaN


In [20]:
ev2 = pd.DataFrame(merged.groupby(["country", "year"])["value"].sum())

In [22]:
merged_final = pd.merge(ev2, co, on=["country", "year"], how="left")
merged_final

,country,year,value,co2
0,Australia,2011,9.800689e+01,403.828773
1,Australia,2012,5.500324e+02,406.150910
2,Australia,2013,8.800386e+02,397.887458
3,Australia,2014,3.200174e+03,393.952876
4,Australia,2015,5.360227e+03,401.793062
...,...,...,...,...
595,United Kingdom,2019,3.262421e+05,364.753306
596,United Kingdom,2020,5.882993e+05,326.263199
597,United Kingdom,2021,1.020273e+06,346.773181
598,United Kingdom,2022,1.320223e+06,NaN


In [24]:
merged_final.dtypes

country     object
year         int64
value      float64
co2        float64
dtype: object

In [28]:
merged_final["value"] = merged_final["value"].astype(int)

In [36]:
merged_final.rename(columns={"co2" : "co2_metric_tons"}, inplace=True)

In [35]:
merged_final.rename(columns={"value" : "ev_sales"}, inplace=True)

In [37]:
merged_final

,country,year,ev_sales,co2_metric_tons
0,Australia,2011,98,403.828773
1,Australia,2012,550,406.150910
2,Australia,2013,880,397.887458
3,Australia,2014,3200,393.952876
4,Australia,2015,5360,401.793062
...,...,...,...,...
595,United Kingdom,2019,326242,364.753306
596,United Kingdom,2020,588299,326.263199
597,United Kingdom,2021,1020273,346.773181
598,United Kingdom,2022,1320222,NaN


In [40]:
merged_final = merged_final.dropna()

In [43]:
merged_final["co2_metric_tons"] = merged_final["co2_metric_tons"].round(2)

/var/folders/3j/f7d6p9t136x34vdmdy30wrrc0000gn/T/ipykernel_17948/1090810860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final["co2_metric_tons"] = merged_final["co2_metric_tons"].round(2)


In [44]:
merged_final

,country,year,ev_sales,co2_metric_tons
0,Australia,2011,98,403.83
1,Australia,2012,550,406.15
2,Australia,2013,880,397.89
3,Australia,2014,3200,393.95
4,Australia,2015,5360,401.79
...,...,...,...,...
593,United Kingdom,2017,178070,387.37
594,United Kingdom,2018,248109,379.73
595,United Kingdom,2019,326242,364.75
596,United Kingdom,2020,588299,326.26


In [45]:
merged_final.to_csv("ev_co2_final.csv")